# Supply chain partitioning example


In this notebook, we will use cudf and cuGraph to show how to read multiple files, compute WCC, and dump to output files

* Created:   01/22/2020

RAPIDS Versions: 0.11.1

Test Hardware
* GV100 32G, CUDA 10.1

Using docker container: rapidsai/rapidsai:cuda10.1-runtime-centos7

## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon. 

In [1]:
# Import needed libraries
import cugraph
import cudf
import os

#### Prep

In [2]:
# Test files
root_dir = '../data/'
output_dir = "../../"
input_file_base = 'supplyChain-data'

In [3]:
def print_components(_df, id):
    
    _f = _df.query('labels == @id')
  
    part = []
    for i in range(len(_f)):
        part.append(_f['org_vertices'][i])
    print(part)

In [4]:
def list_components(_df, id):
    
    _f = _df.query('labels == @id')
  
    part = []
    for i in range(len(_f)):
        part.append(_f['org_vertices'][i])
    
    return part  

#### loop to read files, renumber the node list (if index not starting from 0), create graphs, compute WCC, and dump WCC label info to multiple output files

In [5]:
for i in [1,2]:

    datafile = root_dir+input_file_base+str(i)+'.csv'
    gdf = cudf.read_csv(datafile, delimiter=" ", names=['src', 'dst'], dtype=['int32', 'int32'] )
    
    # renumbering in case the node index does not start from 0
    gdf['renumbered_src'], gdf['renumbered_dst'], mapping = cugraph.renumber(gdf['src'], gdf['dst'])
    
    # create undirectred graph
    gpuG = cugraph.Graph()
    gpuG.from_cudf_edgelist(gdf, source='renumbered_src', target='renumbered_dst')
    wcc = cugraph.weakly_connected_components(gpuG)
    
    # back to original vertices before numbering 
    wcc['org_vertices'] = mapping[wcc['vertices']]
    
    label_gby = wcc.groupby('labels')
    label_count = label_gby.count()
    print("Total number of components found for ", datafile, " : ", len(label_count))
    
    # print out
    tempdf = cudf.DataFrame()
    outputfile = output_dir + input_file_base+str(i) + '_output.csv'
    for j in range(len(label_count)):
        print("Vertex Ids that belong to component label ", j+1, ": ") # wcc output label starting from 1
        print_components(wcc, j+1)
        tempdf['WCC'+str(j+1)] = list_components(wcc, j+1)
    tempdf.to_csv(outputfile,index=False)

Total number of components found for  ../data/supplyChain-data1.csv  :  1
Vertex Ids that belong to component label  1 : 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Total number of components found for  ../data/supplyChain-data2.csv  :  4
Vertex Ids that belong to component label  1 : 
[6, 10, 14, 18]
Vertex Ids that belong to component label  2 : 
[7, 11, 15, 19]
Vertex Ids that belong to component label  3 : 
[8, 12, 16, 20]
Vertex Ids that belong to component label  4 : 
[9, 13, 17, 21]
